In [1]:
import pandas as pd

train_df = pd.read_csv('../../input/feedback-prize-effectiveness/train.csv')

In [2]:
train_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate


In [3]:
from tqdm import tqdm

pred_dfs = []
for essay_id in tqdm(sorted(train_df['essay_id'].unique())):
    tmp_df = train_df[train_df['essay_id']==essay_id].reset_index(drop=True)

    num_ineff = (tmp_df['discourse_effectiveness']=='Ineffective').sum()
    num_ade = (tmp_df['discourse_effectiveness']=='Adequate').sum()
    num_eff = (tmp_df['discourse_effectiveness']=='Effective').sum()
    num_total = len(tmp_df)

    num_ineff, num_ade, num_eff, num_total

    pred_df = tmp_df[['discourse_id']].copy()
    pred_df['Ineffective'] = num_ineff / num_total
    pred_df['Adequate'] = num_ade / num_total
    pred_df['Effective'] = num_eff / num_total

    pred_dfs.append(pred_df)
    
pred_df = pd.concat(pred_dfs, axis=0).reset_index(drop=True)
train_df = train_df.merge(pred_df[['discourse_id','Ineffective','Adequate','Effective']], on='discourse_id', how='left')
train_df['label'] = train_df['discourse_effectiveness'].map({'Ineffective':0, 'Adequate':1, 'Effective':2})

100%|██████████| 4191/4191 [00:13<00:00, 311.94it/s]


In [4]:
import os
os.makedirs('./result', exist_ok=True)
train_df.to_csv('./result/train.csv', index=False)

In [5]:
train_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,Ineffective,Adequate,Effective,label
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,0.111111,0.888889,0.0,1
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,0.111111,0.888889,0.0,1
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,0.111111,0.888889,0.0,1
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,0.111111,0.888889,0.0,1
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,0.111111,0.888889,0.0,1
